# Daily graphs of COVID-19 pandemic

- Automatically updated every day at 22:00 (Argentina Standard Time).

- Data downloaded from [John Hopkins University repository](https://github.com/CSSEGISandData/COVID-19).

- Code released under [GNU GPLv3 License](https://raw.githubusercontent.com/epassaro/covid-19/master/LICENSE) at [github.com/epassaro/covid-19](https://github.com/epassaro/covid-19).

In [1]:
from ruamel.yaml import YAML
import itertools
import numpy as np
import pandas as pd
from bokeh.plotting import figure, show, ColumnDataSource
from bokeh.models.tools import HoverTool
from bokeh.palettes import Category10, Category20
from bokeh.io import output_notebook

output_notebook()

Loading BokehJS ...

In [2]:
!wget -qO time_series_covid19_confirmed_global.csv https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv

In [3]:
yaml = YAML()

with open('plots.yml') as f:
    countries = yaml.load(f)

In [4]:
def create_table(fname, countries, key):
    
    map_names = {'US': 'United States',
                 'Korea, South': 'South Korea'}
   
    df = pd.read_csv(fname)
    df = df.drop(columns=['Province/State', 'Lat', 'Long'])
    df = df.groupby(by=['Country/Region']).sum()
    df = df.transpose()
    df.columns.name = None
    df.index.name = 'date'
    df.index = pd.to_datetime(df.index)
    df = df.reset_index()

    df_list = []
    for country in countries[key]:
        day_zero = df[country].ne(0).idxmax()
        
        d = df[country][day_zero:]
        d = d.reset_index(drop=True)
        
        t = df['date'][day_zero:]
        try: 
            t.name = 'date{}'.format(map_names[country])

        except KeyError:
            t.name = 'date{}'.format(country)
        
        t = t.reset_index(drop=True)
        t = t.map(lambda x: x.strftime('%d-%m-%Y'))
        
        df_list.append(d)
        df_list.append(t)
    
    df = pd.concat(df_list, axis=1)
    df = df.rename(columns=map_names)
    
    
    return df

In [5]:
def plot_cases(df, name):
    
    n = len(df.columns)/2
    if n <= 10:
        palette = itertools.cycle(Category10[n])
    else:
        palette = itertools.cycle(Category20[n])
    
    source = ColumnDataSource(df)
    p = figure(plot_width=680, plot_height=420)

    hovers = []
    for country in df[ [c for c in df.columns if not c.startswith('date') ] ]:
    
        color = next(palette)
        p.line(x='index', y=country, source=source,
               line_width=1.5, 
               legend_label=country, 
               color=color)

        p.circle(x='index', y=country, source=source,
                 legend_label=country, 
                 color=color,
                 name=country) # Tooltip are shown only on points
    
        TOOLTIPS = [('Country', country),
                    ('Day', '@index'),
                    ('Date', '@{{date{}}}'.format(country)),
                    ('Total cases', '@{{{}}}'.format(country))] # Triple curly braces for names containing whitespaces
    
        hover = HoverTool(tooltips=TOOLTIPS, names=[country])
        hovers.append(hover)

    p.add_tools(*hovers)
    p.legend.click_policy = 'hide'
    p.legend.location = 'top_left'
    p.legend.label_text_font_size = '8pt'
    p.yaxis.formatter.use_scientific = False
    p.xaxis.axis_label = "Days since 1st case"
    p.yaxis.axis_label = "Total cases"
    p.title.text = "Cumulative confirmed COVID-19 cases ({})".format(name)

    return show(p)

In [6]:
fname = './time_series_covid19_confirmed_global.csv'

In [7]:
_ = [ plot_cases(create_table(fname, countries, k), k) for k in countries ]